# CSC320: Images in the Frequency Domain

In [ ]:
import numpy as np
import cv2
from scipy import ndimage, misc

%matplotlib notebook
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_file, show
from bokeh.io import show, output_notebook

# use this palette for all plots
from bokeh.palettes import Category10_10 as palette

In [ ]:
def plot_fcns(xs, ys, backend='svg', colors=None, label_x='x', label_y='y'):
    assert len(xs) == len(ys)
    if colors is None:
        colors = palette[0:len(ys)]
        
    p = figure(plot_height=300, plot_width=500, 
               x_range=(-np.pi, np.pi), y_range=(-1.5, 1.5),
               x_axis_label=label_x, y_axis_label=label_y,
               output_backend=backend)
    p.multi_line(xs, ys, color=colors)
    p.xaxis.visible = False
    p.yaxis.visible = False
    p.xgrid.visible = True
    p.ygrid.visible = True
    p.background_fill_color = None
    p.border_fill_color = None 
    output_notebook()
    show(p)

In [ ]:
def plot_fcns_freq(xs, ys, backend='svg', colors=None, label_x='Frequency (Hz)', label_y='Amplitude'):
    assert len(xs) == len(ys)
    if colors is None:
        colors = palette[0:len(ys)]
        
    p = figure(plot_height=300, plot_width=500,
               x_axis_label=label_x, y_axis_label=label_y,
               output_backend=backend)
    p.multi_line(xs, ys, color=colors)
    #for i, (x, y) in enumerate(zip(xs, ys)):
    #    p.step(x, y, color=colors[i])
    p.background_fill_color = None
    p.border_fill_color = None 
    output_notebook()
    show(p)

In [ ]:
def imshow(img):
    plt.figure()
    ax = plt.axes([0,0,1,1])
    plt.imshow(img, interpolation='nearest', cmap='gray')
    plt.axis('off')

In [ ]:
# log avoiding zeroes, replacing them with very small numbers
def nonzero_log(img):
    return np.log(img+np.finfo(np.float32).eps)

## Approximating Functions with Sine Curves

Let's define a simple step function:

In [ ]:
def stepfcn(x):
    if x <0:
        return -1
    return 1

stepfcn_array = np.vectorize(stepfcn)

In [ ]:
x = np.linspace(-np.pi,np.pi,10000) # 100 linearly spaced numbers
y = stepfcn_array(x) # computing the values of our function

In [ ]:
plot_fcns([x], [y])

Our first attempt at approximating the step curve with sin(x)

In [ ]:
plot_fcns([x, x],[y, np.sin(x)])

In [ ]:
# Show how sin function changes with different parameters
plot_fcns([x, x], [np.sin(x), 1.5*np.sin(x)])
plot_fcns([x, x], [np.sin(x), np.sin(2*x)])
plot_fcns([x, x], [np.sin(x), np.sin(x+np.pi/2)])

In [ ]:
# See AZ lectures slides, pg2: http://www.robots.ox.ac.uk/~az/lectures/ia/lect2.pdf
def fourier_component(x, n):
    return 4/((2*n-1)*np.pi)*np.sin((2*n-1)*x)

def fourier_series(x, n=1):
    result = np.zeros_like(x)
    for i in range(1, n+1):
        result += fourier_component(x, n=i)
    return result

In [ ]:
plot_fcns([x, x],[y, fourier_series(x, n=1)])

In [ ]:
plot_fcns([x, x],[y, fourier_series(x, n=2)])

In [ ]:
plot_fcns([x, x, x],[y, fourier_component(x, n=1), fourier_component(x, n=2)])

In [ ]:
plot_fcns([x, x],[np.sin(x), fourier_component(x, n=1)], colors=palette[0:2])
plot_fcns([x],[fourier_component(x, n=2)], colors=palette[1])

In [ ]:
plot_fcns([x, x],[y, fourier_series(x, n=3)])

In [ ]:
plot_fcns([x, x],[y, fourier_series(x, n=10)])

## The Fourier Transform as a Basis

If we try transforming a constant function:

In [ ]:
flat_fcn = np.ones_like(x)
flat_sp = np.absolute(np.fft.fft(flat_fcn))
flat_freq = np.fft.fftfreq(x.shape[-1])

In [ ]:
plot_fcns([x],[flat_fcn])

In [ ]:
plot_fcns_freq([flat_freq],[flat_sp])

Let's try transforming a simple sine curve into the frequency domain using the Fourier transform (actually the Fast Fourier Transform (FFT))

In [ ]:
sin_fcn = np.sin(100*x)
sin_sp = np.absolute(np.fft.fft(sin_fcn))
sin_freq = np.fft.fftfreq(x.shape[-1])

In [ ]:
plot_fcns([x],[sin_fcn])

In [ ]:
plot_fcns_freq([sin_freq],[sin_sp])

Note that in the frequency domain, there is a we see only one peak (actually one negative and one positive), i.e. this function can be represented by a single point in frequency space.

Let's instad try the Fourier transform of a the sum of two sinusoids

In [ ]:
sin_fcn = np.sin(x)+np.sin(10*x)
sin_sp = np.absolute(np.fft.fft(sin_fcn))
sin_freq = np.fft.fftfreq(x.shape[-1])

In [ ]:
plot_fcns([x],[sin_fcn])

In [ ]:
plot_fcns_freq([sin_freq],[sin_sp.real])

Now we see two peaks, i.e. this function can be represented by a 2 points in frequency space.

Let's try something more complex now, let's go back to our step function, and see what the Fourier transform of that looks like

In [ ]:
step_sp = np.absolute(np.fft.fft(y))
step_freq = np.fft.fftfreq(x.shape[-1])

In [ ]:
plot_fcns([x], [y])

In [ ]:
plot_fcns_freq([step_freq],[step_sp])

We can see that the step function requires a lot of points in frequency space to represent!

## 2D Image Fourier Transform Examples

![Example Image](zoomedstreet.jpg)

In [ ]:
img = cv2.imread('zoomedstreet.png', cv2.IMREAD_GRAYSCALE)
imshow(img)

In [ ]:
img_fft = np.fft.fft2(img)
img_fft_centred = np.fft.fftshift(img_fft)

### Frequency

If we just try to plot the magnitude of the frequency components then...

In [ ]:
imshow(np.abs(img_fft_centred))

In [ ]:
imshow(20*np.log(np.abs(img_fft_centred)))

### Phase

In [ ]:
imshow(np.angle(img_fft_centred))

### High-pass Filter

In [ ]:
rows, cols = img.shape
crow, ccol = rows//2 , cols//2
boxsize=50

img_fft_centred_highpass = img_fft_centred.copy()
img_fft_centred_highpass[crow-boxsize:crow+boxsize, ccol-boxsize:ccol+boxsize] = 0
img_fft_highpass = np.fft.ifftshift(img_fft_centred_highpass)
img_back = np.fft.ifft2(img_fft_highpass)
img_back = np.abs(img_back)

In [ ]:
imshow(nonzero_log(np.abs(img_fft_centred_highpass)))

In [ ]:
imshow(img_back)

### Low-pass Filter

In [ ]:
boxsize=50

img_fft_centred_lowpass = np.zeros_like(img_fft_centred)
img_fft_centred_lowpass[crow-boxsize:crow+boxsize, ccol-boxsize:ccol+boxsize] = img_fft_centred[crow-boxsize:crow+boxsize, ccol-boxsize:ccol+boxsize]
img_fft_lowpass = np.fft.ifftshift(img_fft_centred_lowpass)
img_back = np.fft.ifft2(img_fft_lowpass)
img_back = np.abs(img_back)

In [ ]:
imshow(nonzero_log(np.abs(img_fft_centred_lowpass)))

In [ ]:
imshow(img_back)

### DC Component

In [ ]:
img_fft_centred_lowpass = np.zeros_like(img_fft_centred)
img_fft_centred_lowpass[crow, ccol] = img_fft_centred[crow, ccol]
img_fft_lowpass = np.fft.ifftshift(img_fft_centred_lowpass)
img_back = np.fft.ifft2(img_fft_lowpass)
img_back = np.abs(img_back)

In [ ]:
imshow(img_back)

### Gaussian Low-Pass Filter

In [ ]:
ones_fft = np.ones_like(img)
gaussian_fft = ndimage.fourier_gaussian(ones_fft, sigma=10)
gaussian_fft_centred = np.fft.fftshift(gaussian_fft)
imshow(gaussian_fft_centred)

In [ ]:
img_low_pass_fft = img_fft*gaussian_fft
img_back = np.fft.ifft2(img_low_pass_fft)
img_back = np.abs(img_back)

In [ ]:
imshow(nonzero_log(np.abs(np.fft.fftshift(img_low_pass_fft))))

In [ ]:
imshow(img_back)

### Gaussian High-Pass Filter

In [ ]:
ones_fft = np.ones_like(img)
inv_gaussian_fft = ones_fft-ndimage.fourier_gaussian(ones_fft, sigma=10)
inv_gaussian_fft_centred = np.fft.fftshift(inv_gaussian_fft)
imshow(inv_gaussian_fft_centred)

In [ ]:
img_high_pass_fft = img_fft*inv_gaussian_fft
img_back = np.fft.ifft2(img_high_pass_fft)
img_back = np.abs(img_back)

In [ ]:
imshow(nonzero_log(np.abs(np.fft.fftshift(img_high_pass_fft))))

In [ ]:
imshow(img_back)

### Gaussian Band-Pass Filter

In [ ]:
ones_fft = np.ones_like(img)
gaussian_bandpass_fft = ndimage.fourier_gaussian(ones_fft, sigma=5)-ndimage.fourier_gaussian(ones_fft, sigma=10)
gaussian_bandpass_fft_centred = np.fft.fftshift(gaussian_bandpass_fft)
imshow(gaussian_bandpass_fft_centred)

In [ ]:
img_band_pass_fft = img_fft*gaussian_bandpass_fft
img_back = np.fft.ifft2(img_band_pass_fft)
img_back = np.abs(img_back)

In [ ]:
imshow(img_back)